# Classical Portfolio Optimization

In order to perform the optimization, the stock data must be obtained, the expected returns must be calculated, the variance of the stock data must be calculated and the effecient frontier must be found.

This notebook is used to generate a portfolio of 10 random stocks from the AlphaVantage API and then optimize the portfolio weights using Markowitz theory. This portfolio will be used as the baseline in which the analysis con

## Obtaining stock data



In [0]:
test commit from colab